# IMDB Sentiment Classification With LSTM

Sentiment classification with LSTM on IMDB dataset. 

This is an example that use LSTM as a fix-length layer on neural networks (Another is usage of LSTM or RNN is var-length, see [stateful_lstm](https://github.com/fchollet/keras/blob/master/examples/stateful_lstm.py))

More information about LSTM in Keras: [LSTM intructions](https://keras.io/layers/recurrent/)

More information about IMDB datasets: [IMDB_datasets](https://github.com/autoliuweijie/DeepLearning/blob/master/deep_learning_with_python/dlwp/c22_project_predict_sentiment_with_movie_review.ipynb)

## 1. Preparation

Import classes and functions we will use.

In [24]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb

In [2]:
# fix random seed
seed = 7
np.random.seed(seed)

In [11]:
# Set some global variable
max_features = 20000  # the total number of words
maxlen = 80
batch_size = 32

Load datasets. Every items in X_train is a list which means a sentence whose words has been replaced by int numbers.

In [9]:
# load data
(X_train, Y_train), (X_val, Y_val) = imdb.load_data(nb_words=max_features)
print(X_train.shape)
print(X_train[0])
print("X type: %s"%(type(X_train[0])))
print(Y_train.shape)
print(Y_train[0])

(25000,)
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
X type: <type 'list'>
(25000,)
1


In [10]:
# pad sequences to fix length
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_val = sequence.pad_sequences(X_val, maxlen=maxlen)
print(len(X_train[0]))

80


## 2. Build Model

In [28]:
# define a model
def create_model():
    model = Sequential()
    model.add(Embedding(max_features, 32, input_length=maxlen, dropout=0.2))  # use 32-D vector to represent a word
    model.add(LSTM(64, dropout_W=0.2, dropout_U=0.2, stateful=False, return_sequences=True))  # return_sequences means output in sequence, not average?
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

model = create_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_13 (Embedding)         (None, 80, 32)        640000      embedding_input_13[0][0]         
____________________________________________________________________________________________________
lstm_13 (LSTM)                   (None, 80, 64)        24832       embedding_13[0][0]               
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 5120)          0           lstm_13[0][0]                    
____________________________________________________________________________________________________
dense_12 (Dense)                 (None, 1)             5121        flatten_3[0][0]                  
Total params: 669953
______________________________________________________________________

In [31]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_val, Y_val), verbose=1)

INFO (theano.gof.compilelock): Refreshing lock /Users/liuweijie/.theano/compiledir_Darwin-16.1.0-x86_64-i386-64bit-i386-2.7.12-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /Users/liuweijie/.theano/compiledir_Darwin-16.1.0-x86_64-i386-64bit-i386-2.7.12-64/lock_dir/lock


Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 61s - loss: 0.4801 - acc: 0.7557 - val_loss: 0.3534 - val_acc: 0.8430
Epoch 2/15
25000/25000 [==============================] - 63s - loss: 0.3166 - acc: 0.8660 - val_loss: 0.3510 - val_acc: 0.8447
Epoch 3/15
25000/25000 [==============================] - 64s - loss: 0.2578 - acc: 0.8951 - val_loss: 0.3660 - val_acc: 0.8376
Epoch 4/15
25000/25000 [==============================] - 64s - loss: 0.1956 - acc: 0.9220 - val_loss: 0.4119 - val_acc: 0.8315
Epoch 5/15
25000/25000 [==============================] - 65s - loss: 0.1421 - acc: 0.9455 - val_loss: 0.4513 - val_acc: 0.8212
Epoch 6/15
25000/25000 [==============================] - 65s - loss: 0.1078 - acc: 0.9603 - val_loss: 0.5206 - val_acc: 0.8212
Epoch 7/15
25000/25000 [==============================] - 65s - loss: 0.0861 - acc: 0.9681 - val_loss: 0.5634 - val_acc: 0.8146
Epoch 8/15
25000/25000 [==============================

In [32]:
# evaluate model
scores = model.evaluate(X_val, Y_val)
print("Val_acc: %.2f%%"%(scores[1]*100))

24992/25000 [============================>.] - ETA: 0sVal_acc: 80.40%
